In [12]:
import json

import numpy as np
from datasets import load_dataset
from math_agent import (
    MathAgent,
    MathEnvironment,
    extract_result_value,
    solve_task,
)
from termcolor import colored
from tqdm import tqdm

from tapeagents.llms import TrainableLLM


In [13]:
env = MathEnvironment()


def eval(tested_agent, test_set, name="") -> float:
    test_solved = []
    n = 0
    for sample in tqdm(test_set):
        sample = extract_result_value(sample)
        try:
            tape = solve_task(tested_agent, env, sample)
            test_solved.append(int(tape.metadata.result["solved"]))
        except Exception as e:
            print(colored("Failed to solve task: {e}", "red"))
            test_solved.append(0)
            raise e
        acc = np.mean(test_solved).item()
        n = len(test_solved)
        if n % 10 == 0 and n > 0:
            print(f"{n}: Current accuracy: {acc:.3f}")
            with open("results.jsonl", "a") as f:
                f.write(json.dumps({name: acc, "n": n}) + "\n")
    acc = np.mean(test_solved).item()
    with open("results.jsonl", "a") as f:
        f.write(json.dumps({name: acc, "n": n}) + "\n")
    return acc


In [3]:
test_dataset = load_dataset("openai/gsm8k", "main", split="test")
test_samples = [s for s in test_dataset]
np.random.seed(42)
np.random.shuffle(test_samples)  # type: ignore
test_set = test_samples[:200]

dataset = load_dataset("openai/gsm8k", "main", split="train")
val_samples = [s for s in dataset]
np.random.seed(42)
np.random.shuffle(val_samples)  # type: ignore
val_set = val_samples[:200]


## Untuned model accuracy

In [4]:
# run inference: vllm serve meta-llama/Meta-Llama-3.1-8B-Instruct
untuned_agent = MathAgent(
    llms={
        "default": TrainableLLM(
            base_url="http://localhost:8000",
            model_name="meta-llama/Meta-Llama-3.1-8B-Instruct",
            tokenizer_name="meta-llama/Meta-Llama-3.1-8B-Instruct",
            parameters=dict(temperature=0.1),
            use_cache=False,
        )
    }
)


In [12]:
acc = eval(untuned_agent, test_set)
print(f"Untuned on test {acc:.3f}")


  6%|▌         | 11/200 [00:50<15:08,  4.81s/it]

10: Current accuracy: 0.545


 10%|█         | 21/200 [01:22<08:29,  2.85s/it]

20: Current accuracy: 0.476


 16%|█▌        | 31/200 [01:55<08:12,  2.91s/it]

...

180: Current accuracy: 0.663


 96%|█████████▌| 191/200 [15:29<00:35,  3.95s/it]

190: Current accuracy: 0.654


100%|██████████| 200/200 [16:12<00:00,  4.86s/it]

Untuned on test 0.660


## Tuned model accuracy

In [7]:
# run inference: vllm serve gsm8k/tuning/llama31_70b_train_t02/tune1/intermediate/800/
tuned_agent = MathAgent(
    llms={
        "default": TrainableLLM(
            base_url="http://localhost:8000",
            model_name="gsm8k/tuning/llama31_70b_train_t02/tune1/intermediate/800/",
            tokenizer_name="meta-llama/Meta-Llama-3.1-8B-Instruct",
            parameters=dict(temperature=0.0),
            use_cache=False,
        )
    }
)


In [8]:
tuned_acc = eval(tuned_agent, test_set, "tuned_acc")
print(f"Tuned on test {tuned_acc:.3f}")


  6%|▌         | 11/200 [00:54<16:13,  5.15s/it]

10: Current accuracy: 0.727


 10%|█         | 21/200 [01:43<13:07,  4.40s/it]

20: Current accuracy: 0.762


 16%|█▌        | 31/200 [02:23<11:41,  4.15s/it]

...

180: Current accuracy: 0.768


 96%|█████████▌| 191/200 [21:43<00:51,  5.74s/it]

190: Current accuracy: 0.775


100%|██████████| 200/200 [22:35<00:00,  6.78s/it]

Tuned on test 0.775


In [14]:
# check teacher model
big_llm = TrainableLLM(
    base_url="https://api.together.xyz",
    model_name="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    tokenizer_name="meta-llama/Meta-Llama-3.1-70B-Instruct",
    parameters=dict(temperature=0.2),
    use_cache=False,
)
big_agent = MathAgent(llms={"default": big_llm})


In [15]:
big_acc = eval(big_agent, test_set, "big_acc")
print(f"Teacher on test {big_acc:.3f}")


  5%|▌         | 10/200 [01:59<38:17, 12.09s/it]

10: Current accuracy: 0.800


 10%|█         | 20/200 [04:12<39:39, 13.22s/it]

20: Current accuracy: 0.900


 15%|█▌        | 30/200 [05:27<18:59,  6.70s/it]

...

180: Current accuracy: 0.933


 95%|█████████▌| 190/200 [36:17<01:27,  8.75s/it]

190: Current accuracy: 0.937


100%|██████████| 200/200 [37:49<00:00, 11.35s/it]

200: Current accuracy: 0.935
Teacher on test 0.935
